<h1 align='center'>Perceptual Losses for Real-Time Style Transfer and Super-Resolution </center>
https://arxiv.org/abs/1603.08155

Play with VGG 

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D
from keras import applications

In [11]:
model = applications.VGG16(include_top=False,
                           weights='imagenet')


In [17]:
layer_dict=dict([(layer.name,layer) for layer in model.layers])

In [18]:
layer_dict

{'block1_conv1': <keras.layers.convolutional.Conv2D at 0x7f3b5c7be950>,
 'block1_conv2': <keras.layers.convolutional.Conv2D at 0x7f3b5c543fd0>,
 'block1_pool': <keras.layers.pooling.MaxPooling2D at 0x7f3b5c7be750>,
 'block2_conv1': <keras.layers.convolutional.Conv2D at 0x7f3b5c7becd0>,
 'block2_conv2': <keras.layers.convolutional.Conv2D at 0x7f3b5c55cad0>,
 'block2_pool': <keras.layers.pooling.MaxPooling2D at 0x7f3b5c5bd150>,
 'block3_conv1': <keras.layers.convolutional.Conv2D at 0x7f3b5c5d0f90>,
 'block3_conv2': <keras.layers.convolutional.Conv2D at 0x7f3b5c5e8e10>,
 'block3_conv3': <keras.layers.convolutional.Conv2D at 0x7f3b5c585750>,
 'block3_pool': <keras.layers.pooling.MaxPooling2D at 0x7f3b5c4f5f90>,
 'block4_conv1': <keras.layers.convolutional.Conv2D at 0x7f3b5c50c910>,
 'block4_conv2': <keras.layers.convolutional.Conv2D at 0x7f3b5c51ff90>,
 'block4_conv3': <keras.layers.convolutional.Conv2D at 0x7f3b5c531610>,
 'block4_pool': <keras.layers.pooling.MaxPooling2D at 0x7f3b5c4e7fd